In [ ]:
model_name = '/data3/gkook/model/Qwen2-Audio-7B-Instruct'

In [ ]:
import os
import pandas as pd
from datasets import Dataset, concatenate_datasets
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, IterableDataset
import random
import librosa
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
# --- 0. 환경 변수 설정 ---
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ["HF_HOME"] = "/data1/jc/AGI"

# Set the base output directory for all saves
BASE_OUTPUT_DIR = "/data1/jc/AGI/LibriSpeech/LibriSpeech/train/LibriSpeech/"
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True) # Ensure the directory exists

# W&B API 키 및 프로젝트 이름 설정
os.environ["WANDB_API_KEY"] = "ed10e1d235ad82c9e6a4dc4dbc622488d71c8ef6"
os.environ["WANDB_PROJECT"] = "qwen2-audio-finetune" # This remains the W&B project name
os.environ["WANDB_DIR"] = BASE_OUTPUT_DIR # Set WANDB_DIR to control where W&B files are stored
print(f"WandB directory set to: {os.environ['WANDB_DIR']}")

print("환경 설정 완료.")

# --- 1. 모델 및 프로세서 로드 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"사용할 디바이스: {device}")

try:
    print(f"Loading Qwen2-Audio-7B-Instruct processor...")
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct", trust_remote_code=True)
    
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    )
    
    print(f"Loading Qwen2-Audio-7B-Instruct model with 4-bit quantization...")
    model = Qwen2AudioForConditionalGeneration.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )
    print("Qwen2-Audio-7B-Instruct 모델 및 프로세서 로드 완료.")
except Exception as e:
    print(f"\n[오류 발생] 모델 로드 중 오류: {e}")
    exit()


In [ ]:
model.num_parameters()

In [ ]:
print_trainable_param = 0
for n , p in model.named_parameters():
    if p.requires_grad==True:
        print('trainable', n, p.numel())
        print_trainable_param += p.numel()
    else:
        print('not trainable', n, p.numel())
print(f"Trainable parameters: {print_trainable_param}")

In [ ]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

count_trainable_parameters(model)

In [ ]:
model.num_parameters()

In [12]:
import datasets
from tqdm import tqdm
import math

def calculate_total_duration(dataset_name):
    """
    Hugging Face 데이터셋의 모든 오디오 파일 총 길이를 계산합니다.
    오류가 발생하는 파일은 건너뛰고 횟수를 셉니다.
    """
    print(f"'{dataset_name}' 데이터셋 로드 중... (시간이 걸릴 수 있습니다)")
    
    try:
        dataset = datasets.load_dataset(dataset_name)
    except Exception as e:
        print(f"데이터셋 로드 중 오류 발생: {e}")
        return None, 0

    total_seconds = 0.0
    error_count = 0
    
    print("데이터셋의 각 split에서 오디오 길이 계산 중...")

    for split_name in dataset.keys():
        print(f"--- {split_name} split 처리 중 ---")
        
        split_dataset = dataset[split_name]
        
        # 첫 번째 예제로 metadata 구조 확인 (디버깅용)
        debug_checked = False
        
        # tqdm을 사용하여 진행 상황 표시
        for idx, example in enumerate(tqdm(split_dataset, desc=f"{split_name} split 진행률")):
            try:
                # 오디오 키가 있는지 확인
                if "audio" not in example:
                    error_count += 1
                    continue
                
                # 오디오 데이터 접근 시도 (디코딩 오류가 여기서 발생할 수 있음)
                audio_data = example.get("audio")
                if audio_data is None:
                    error_count += 1
                    continue
                
                # 첫 번째 예제의 metadata 구조 확인 (디버깅용)
                if not debug_checked and not isinstance(audio_data, dict) and hasattr(audio_data, 'metadata'):
                    print(f"\n[디버깅] 첫 번째 예제의 audio_data 타입: {type(audio_data)}")
                    print(f"[디버깅] metadata 타입: {type(audio_data.metadata)}")
                    if audio_data.metadata is not None:
                        if isinstance(audio_data.metadata, dict):
                            print(f"[디버깅] metadata (dict): {audio_data.metadata}")
                        else:
                            print(f"[디버깅] metadata 속성들: {dir(audio_data.metadata)}")
                            if hasattr(audio_data.metadata, '__dict__'):
                                print(f"[디버깅] metadata.__dict__: {audio_data.metadata.__dict__}")
                    debug_checked = True
                
                # audio_data가 딕셔너리인지 확인
                if isinstance(audio_data, dict):
                    # 일반 딕셔너리 형태인 경우
                    if "array" not in audio_data or "sampling_rate" not in audio_data:
                        error_count += 1
                        continue
                    
                    num_samples = len(audio_data["array"])
                    sampling_rate = audio_data["sampling_rate"]
                    
                    if sampling_rate > 0 and num_samples > 0:
                        duration = num_samples / sampling_rate
                        total_seconds += duration
                    else:
                        error_count += 1
                else:
                    # AudioDecoder 객체인 경우 (datasets.features._torchcodec.AudioDecoder)
                    duration = None
                    sampling_rate = None
                    
                    # 0. AudioDecoder를 딕셔너리처럼 접근 시도 (일부 구현에서는 가능)
                    try:
                        if hasattr(audio_data, '__getitem__'):
                            # 'duration' 키로 직접 접근 시도
                            try:
                                duration = audio_data['duration']
                            except (KeyError, TypeError):
                                pass
                            # 'sampling_rate' 또는 'sample_rate' 키로 접근 시도
                            try:
                                sampling_rate = audio_data.get('sampling_rate') or audio_data.get('sample_rate')
                            except (AttributeError, TypeError):
                                pass
                    except:
                        pass
                    
                    # 1. metadata에서 duration 정보 확인 (가장 빠름, 디코딩 불필요)
                    if duration is None and hasattr(audio_data, 'metadata') and audio_data.metadata is not None:
                        metadata = audio_data.metadata
                        # metadata가 딕셔너리인 경우
                        if isinstance(metadata, dict):
                            duration = metadata.get('duration')
                            sampling_rate = metadata.get('sample_rate') or metadata.get('sampling_rate')
                        # metadata가 객체인 경우 - 다양한 속성명 시도
                        else:
                            # duration 속성 시도 (torchcodec의 AudioStreamMetadata는 duration_seconds_from_header 사용)
                            duration = getattr(metadata, 'duration_seconds_from_header', None)
                            if duration is None:
                                duration = getattr(metadata, 'duration', None)
                            if duration is None:
                                # 다른 가능한 속성명들 시도
                                duration = getattr(metadata, 'length', None)
                                duration = getattr(metadata, 'duration_seconds', None) if duration is None else duration
                            
                            # sampling_rate 속성 시도 (torchcodec의 AudioStreamMetadata는 sample_rate 사용)
                            sampling_rate = getattr(metadata, 'sample_rate', None)
                            if sampling_rate is None:
                                sampling_rate = getattr(metadata, 'sampling_rate', None)
                            if sampling_rate is None:
                                sampling_rate = getattr(metadata, 'sr', None)
                    
                    # metadata가 없거나 duration을 못 찾은 경우, __dict__에서 직접 확인
                    if duration is None and hasattr(audio_data, 'metadata'):
                        # metadata를 dict로 변환 시도 (torchcodec의 AudioStreamMetadata는 __dict__에 duration_seconds_from_header가 있음)
                        try:
                            if hasattr(audio_data.metadata, '__dict__'):
                                metadata_dict = audio_data.metadata.__dict__
                                duration = metadata_dict.get('duration_seconds_from_header') or metadata_dict.get('duration') or metadata_dict.get('length')
                                sampling_rate = metadata_dict.get('sample_rate') or metadata_dict.get('sampling_rate') or metadata_dict.get('sr')
                        except:
                            pass
                    
                    # 2. duration이 있으면 바로 사용
                    if duration is not None and duration > 0:
                        total_seconds += duration
                        continue
                    
                    # 3. duration이 없으면 sampling_rate 확인 후 get_all_samples()로 계산
                    if not sampling_rate:
                        # _desired_sample_rate에서 가져오기
                        if hasattr(audio_data, '_desired_sample_rate') and audio_data._desired_sample_rate:
                            sampling_rate = audio_data._desired_sample_rate
                    
                    # 4. get_all_samples()로 샘플 수 계산 (FFmpeg 디코딩 오류 발생 가능)
                    if sampling_rate:
                        try:
                            samples = audio_data.get_all_samples()
                            # samples가 torch tensor인 경우
                            if hasattr(samples, 'data'):
                                num_samples = samples.data.shape[-1] if len(samples.data.shape) > 0 else len(samples.data)
                            # samples가 numpy array인 경우
                            elif hasattr(samples, 'shape'):
                                num_samples = samples.shape[-1] if len(samples.shape) > 0 else len(samples)
                            # 일반 리스트나 배열인 경우
                            else:
                                num_samples = len(samples) if hasattr(samples, '__len__') else 0
                            
                            if num_samples > 0 and sampling_rate > 0:
                                duration = num_samples / sampling_rate
                                total_seconds += duration
                            else:
                                error_count += 1
                        except Exception:
                            # get_all_samples() 호출 시 에러 발생 (FFmpeg 디코딩 오류 등)
                            error_count += 1
                    else:
                        # sampling_rate도 없으면 에러
                        error_count += 1
                
            except (RuntimeError, KeyError, TypeError, AttributeError, ValueError) as e:
                # 'The frame has 0 channels' 같은 모든 디코딩 오류를 여기서 처리
                error_count += 1
                # 디버깅을 위해 가끔 에러 메시지 출력 (너무 많이 출력되지 않도록)
                if error_count <= 5 or error_count % 1000 == 0:
                    print(f"\n[경고] 오디오 처리 오류 (누적 {error_count}개): {type(e).__name__}")
            except Exception as e:
                # 예상치 못한 다른 에러들
                error_count += 1
                if error_count <= 5:
                    print(f"\n[경고] 예상치 못한 오류 (누적 {error_count}개): {type(e).__name__}: {str(e)[:100]}")

    return total_seconds, error_count

def format_duration(total_seconds):
    """
    총 초를 (시, 분, 초) 형식으로 변환합니다.
    """
    total_seconds = math.floor(total_seconds)
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    return hours, minutes, seconds

# --- 메인 코드 실행 ---
dataset_id = "iknow-lab/BridgeDataV2-audio"
total_duration_seconds, errors = calculate_total_duration(dataset_id)

if total_duration_seconds is not None:
    h, m, s = format_duration(total_duration_seconds)
    print("\n" + "="*30)
    print(f"'{dataset_id}' 데이터셋의 총 오디오 길이 (유효한 파일 기준):")
    print(f"==> {h}시간 {m}분 {s}초")
    print(f"(총 {total_duration_seconds:.2f} 초)")
    print("\n" + f"총 {errors}개의 오디오 파일 처리 중 오류가 발생하여 건너뛰었습니다.")
    print("="*30)
    print("="*30)

'iknow-lab/BridgeDataV2-audio' 데이터셋 로드 중... (시간이 걸릴 수 있습니다)
데이터셋의 각 split에서 오디오 길이 계산 중...
--- train split 처리 중 ---


train split 진행률:   0%|          | 10/21676 [00:00<03:48, 94.83it/s]


[디버깅] 첫 번째 예제의 audio_data 타입: <class 'datasets.features._torchcodec.AudioDecoder'>
[디버깅] metadata 타입: <class 'torchcodec._core._metadata.AudioStreamMetadata'>
[디버깅] metadata 속성들: ['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'begin_stream_seconds_from_header', 'bit_rate', 'codec', 'duration_seconds_from_header', 'num_channels', 'path', 'sample_format', 'sample_rate', 'stream_index']
[디버깅] metadata.__dict__: {'duration_seconds_from_header': 1.591293, 'begin_stream_seconds_from_header': None, 'bit_rate': 352800.0, 'codec': 'pcm_s16le', 'stream_index': 0, 'sample_rate': 22050, 'num_channels': 1,

train split 진행률: 100%|██████████| 21676/21676 [02:23<00:00, 150.80it/s]


'iknow-lab/BridgeDataV2-audio' 데이터셋의 총 오디오 길이 (유효한 파일 기준):
==> 17시간 5분 28초
(총 61528.57 초)

총 0개의 오디오 파일 처리 중 오류가 발생하여 건너뛰었습니다.
